In [34]:
import numpy as np
#####################################
### Functions for ICC calculation ###
#####################################

def icc(g, e):
    """Calculates IntraClass Correlation

    Args:
        g (float): Sigma^2_g estimated variance for random effect(s)
        e (float): Sigma^2_e estimated variance for residual

    Returns:
        float: ICC
    """
    ICC = g/(e+g)
    print(f"*** ICC = {ICC:.4f} ***")
    return ICC

sigma_g = 15.8758 # group variance(s), if multiple, add up to single float 
sigma_e = 10.5788 # residual variance
icc(sigma_g, sigma_e)

*** ICC = 0.6001 ***


0.6001149138524113

In [36]:
###########################################################
#### Function to calculate confidence interval for ICC ####
###########################################################
from scipy.stats import f
def confidence_interval_icc(MS_b, MS_w, df_b, df_w, n, alpha=0.05):
    """Calculate confidence intervals for ICC

    Args:
        MS_b (float): Mean Squares between (random effect in ANOVA table)
        MS_w (float): Mean Squares within (residual in ANOVA table)
        df_b (int): ANOVA table degrees of freedom, random effects
        df_w (int): ANOVA table degrees of freedom, residual
        n (float): Constant in ANOVA table, expected mean squares
        alpha (float, optional): CI, usually 95%. Defaults to 0.05.
    """
    F = MS_b/MS_w 
    F_u = f.ppf(1-alpha/2, dfn=df_b, dfd=df_w)
    F_l = f.ppf(alpha/2, dfn=df_b, dfd=df_w)
    lower = (F/F_u - 1)/(F/F_u + n - 1)
    upper = (F/F_l - 1)/(F/F_l + n - 1)
    print(f"*** Confidence interval: {lower:.4f} - {upper:.4f} ***")


MS_b = 40.43722  ## Mean Squares between groups (random effects in anova table)
MS_w = 10.57878  ## Mean squares within group (residual in anova table)
c_n = 1.8808 # constant in anova table, expected mean squares
DF_b = 478 # anova table, random effect(s)
DF_w = 421 # anova table, residual

confidence_interval_icc(MS_b, MS_w, DF_b, DF_w, c_n)

*** Confidence interval: 0.5361 - 0.6568 ***


In [42]:
###############################
#### Likelihood Ratio Test ####
###############################
from logging import warning
from scipy.stats import chi2

def likelihood_ratio_test(theta, theta_0, df):
    LRT = theta_0 - theta 
    p = 1-chi2.cdf(LRT, df)
    print(f"*** P-value from Chi-Square: {p:.4f} *** \n *** If below 0.05, reject null hypothesis ***")
    if p < 1e-4:
        warning(" P-value extremely low!")
theta_ll = 5359.6 
theta_0_ll = 5374.7 
diff_params = 6
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)


*** P-value from Chi-Square: 0.0195 *** 
 *** If below 0.05, reject null hypothesis ***


In [43]:
#### Model fitting, check for covariance structure
theta_ll = 29097.6
theta_0_ll = 29136.8 
diff_params = 1
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)

*** P-value from Chi-Square: 0.0000 *** 
 *** If below 0.05, reject null hypothesis ***


In [46]:
### Model selection: Fixed effects, using ML estimation!
theta_ll = 29062.8
theta_0_ll = 29064.1
diff_params = 1
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)

*** P-value from Chi-Square: 0.2542 *** 
 *** If below 0.05, reject null hypothesis ***


In [22]:
#############################################
#### Minimize variance for time profiles ####
#############################################

def minimze_variability(tau_0, tau_1, off_diag, s2_r):
    rho = off_diag/(np.sqrt(tau_0)*np.sqrt(tau_1))
    min_var = (1-rho**2)*tau_0 + s2_r 
    min_t = -rho*np.sqrt(tau_0)/np.sqrt(tau_1)
    print(f"*** Minimum variance: {min_var:.3f} *** \n*** Minimum variance achieved at time point {min_t:.3f} ***")

In [24]:
### Mock exam exercise 3b
t2_0 = 89.7452 # tau0^2
t2_1 = 0.4232 # tau1^2
off_diag = 1.4801 # off-diagonal value
s2_r = 29.1645 # sigma_r^2
minimze_variability(t2_0, t2_1, off_diag, s2_r)

*** Minimum variance: 113.733 *** 
*** Minimum variance achieved at time point -3.497 ***


In [30]:
### T-distribution with high degrees of freedom = normal effect
from scipy.stats import norm, t 
norm.ppf(0.95), t.ppf(0.95, 2654)

(1.6448536269514722, 1.6454279692291978)

# Exercises

### Exercise 3

In [37]:
## Exercise A3
Sigma_g, Sigma_e = 7.765, 31.613
MS_b = 172.899
MS_w = 31.614
df_b = 196
df_w = 3390
c_n = 18.193

icc(Sigma_g, Sigma_e)
confidence_interval_icc(MS_b, MS_w, df_b, df_w, c_n)


*** ICC = 0.1972 ***
*** Confidence interval: 0.1614 - 0.2408 ***


In [52]:
#########################################
#### Sum of Variance components + CI ####
#########################################
from scipy.stats import chi2

def sum_of_var(Sigma_g, Sigma_e, MS_b, MS_w, cn, df_b, df_w, alpha=0.05):
    """Calculate total variance and confidence intervals

    Args:
        Sigma_g (float): VC for between group variance
        Sigma_e (float): VC for within group variance (residual)
        MS_b (float): Mean Squares between from ANOVA table
        MS_w (float): Mean Squares within (residual) from ANOVA table
        cn (float): Constant from expected mean squares column
        df_b (int): Degrees of freedom between groups from ANOVA table
        df_w (int): Degrees of freedom within groups from ANOVA table
        alpha (float, optional): Default 95% confidence for intervals. Defaults to 0.05.
    """
    ### Total variance estimate
    Sigma_t = Sigma_g + Sigma_e 
    ### Variance of total variance estimate
    V_sigma_t = ((2*MS_b**2)/(cn**2 * df_b)) + ((2*(cn - 1)**2 * MS_w**2)/(cn**2 * df_w))
    ### Satterthwaite degrees of freedom
    df_t = 2 * ( (Sigma_t**2) / V_sigma_t )
    U_chi2 = chi2.ppf((1-alpha/2), df_t)
    L_chi2 = chi2.ppf(alpha/2, df_t)
    LCL = (df_t*Sigma_t)/(U_chi2)
    UCL = (df_t*Sigma_t)/(L_chi2)
    print(f"*** Total Variability: {Sigma_t:.2f} ***")
    print(f"*** Confidence interval {LCL:.2f} - {UCL:.2f} ***")    

In [53]:
sum_of_var(Sigma_g, Sigma_e, MS_b, MS_w, c_n, df_b, df_w)

*** Total Variability: 39.38 ***
*** Confidence interval 37.12 - 41.85 ***
